In [22]:
import pandas as pd
import os
import sqlalchemy as sa
import numpy as np

In [23]:
pd.set_option('display.max_columns', None)

In [24]:
db_user = os.getenv('SQL_USER')
db_host = os.getenv('SQL_HOST')
db_pwd = os.getenv('SQL_PWD')

In [25]:
engine = sa.create_engine(f'mssql+pyodbc://{db_user}:{db_pwd}@{db_host}/wrangling?driver=ODBC+Driver+17+for+SQL+Server')

In [26]:
df = pd.read_sql_table('temp_final', engine, 'dbo')

In [27]:
display(df)

,ssn,first_name,middle_name,last_name,gender,age_at_consultation,birth_date,medicare_number,street_address,suburb,state,postcode,phone,email,marital_status,height,weight,bmi,blood_pressure,cholesterol_level,smoking_status,clinical_notes,consultation_timestamp,education,occupation,salary,credit_card_number,years_of_experience,employment_timestamp,medical_rec_id,education_rec_id
0,a100050102,justin,elton,verga,m,36,1980-06-07,8250 99889 1 2,40 marcus clarke street canberra house,canberra city,act,2601,02 5920 4201,verga.justin@mail.com,married,191,67,18,76,138,0,urinary-tract-infection,2016-01-26 06:44:00+00:00,certificate-iv,other-worker,78359,7426 3365 3452 2350,21,2020-04-15 13:24:00+00:00,rec-86693,r15002000
1,a100091149,christopher,jerome,herring,m,21,1995-10-20,2939 34141 2 3,17 inner harbour drive marina palms,patterson lakes,vic,3197,03 6566 8753,christopher.herring@mail.com.au,married-de-facto,174,108,35,77,160,0,back-pain,2016-07-18 13:58:00+00:00,certificate-iii,ict-technician,104897,7356 0165 1634 7804,6,2020-06-13 01:21:00+00:00,rec-10443,r90040119
2,a100222723,billy,david,ross,m,15,1998-05-17,5417 95547 2 3,41 cavenagh street plaza bldg,darwin,nt,0800,08 4559 6907,ross94@mail.com.au,not-applicable,176,73,23,79,191,0,insomnia,2013-03-26 21:59:00+00:00,secondary-school,machinary-driver,14251,5837 2758 2366 5640,3,2020-04-05 06:45:00+00:00,rec-71086,r23272200
3,a100248307,howard,stephen,turner,m,36,1985-01-09,6472 28876 1 1,1351 sidonia road balkebah,baynton,vic,3444,03 4442 1139,howard.turner@gmail.com,married,187,128,36,74,132,0,insomnia,2020-07-23 06:51:00+00:00,certificate-ii,personal-services-worker,78420,3951 5724 9004 2669,15,2019-09-27 12:53:00+00:00,rec-56402,r20407803
4,a100317978,keith,andrew,darr,m,14,2002-07-01,4693 25183 1 2,4 morris way jerrabung,bunyip,vic,3815,03 8293 5481,darr60@mail.com.au,not-applicable,157,45,18,70,139,0,shuffling-gait,2016-08-17 21:12:00+00:00,senior-secondary-school,casual-worker,6909,8247 4592 3961 9751,1,2020-01-24 07:11:00+00:00,rec-674,r97030187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16060,i199840951,evan,maston,bruno,m,23,1996-04-03,4352 69591 2 3,42 borenore lane kenvale,borenore,nsw,2800,02 1588 8711,bruno50@hotmail.com,married,194,75,19,74,181,0,depressed-mood,2018-12-19 16:05:00+00:00,advance-diploma,art-media-professional,103367,4496 1676 9466 2010,5,2020-04-02 07:41:00+00:00,rec-96364,r40995189
16061,i199919157,jillian,erin,perell,f,33,1984-11-22,3005 36572 1 1,3 collins close masonic vlge,lake haven,nsw,2263,02 9404 9177,perell.jillian@mail.com,married,199,84,21,76,140,0,flu,2017-08-18 00:14:00+00:00,advance-diploma,education-professional,112247,9392 1545 1088 9225,17,2020-02-16 06:38:00+00:00,rec-6949,r19995719
16062,i199940217,martha,renee,jackson,f,15,2000-05-11,8818 54716 2 2,22 mitchell street central specialist clinic,darwin,nt,0800,08 6383 0084,jackson.martha@mail.com.au,not-applicable,177,61,19,65,218,0,leg-injury,2015-07-11 01:52:00+00:00,certificate-iii,automotive-worker,91101,8278 6665 0763 8863,1,2020-05-20 04:18:00+00:00,rec-2966,r97914920
16063,i199953012,thomas,joseph,petty,m,27,1989-04-22,9821 42905 2 2,20 dalley street st vincents private hospital,lismore,nsw,2480,02 5323 4883,petty88@aol.com,widowed,176,140,45,78,266,1,cataract-right-eye-(worsening),2016-06-08 08:10:00+00:00,advance-diploma,ict-professional,87913,5666 8118 7243 3499,12,2020-04-17 18:38:00+00:00,rec-67404,r91953920


In [28]:
# Validate if the suburb entered matches the postcode entered (0 = no, 1 = yes, invalid = postcode doesn't exist)
pc = pd.read_csv('postcodes.csv', dtype=str)
pc = pc[['postcode', 'locality', 'state']]
display(pc)

,postcode,locality,state
0,0200,ANU,ACT
1,0200,Australian National University,ACT
2,0800,DARWIN,NT
3,0801,DARWIN,NT
4,0804,PARAP,NT
...,...,...,...
18436,9013,BRISBANE,QLD
18437,9015,BRISBANE,QLD
18438,9464,NORTHGATE MC,QLD
18439,9726,GOLD COAST MC,QLD


In [29]:
pc1 = pc.groupby('postcode')['locality'].apply(list)
df = df.replace(np.nan, '', regex=True)

In [30]:
display(pc1)

postcode
0200                [ANU, Australian National University]
0800                                             [DARWIN]
0801                                             [DARWIN]
0804                                              [PARAP]
0810    [ALAWA, BRINKIN, CASUARINA, COCONUT GROVE, JIN...
                              ...                        
9013                                           [BRISBANE]
9015                                           [BRISBANE]
9464                                       [NORTHGATE MC]
9726                                      [GOLD COAST MC]
9999                                         [NORTH POLE]
Name: locality, Length: 3167, dtype: object

In [31]:
def pcode_valid(row):
    if row['postcode'] != '' and row['suburb'] != '':
        try:
            pcode_src = row['postcode']
            suburb_src = row['suburb']
            suburbs_lookup = [x.lower() for x in pc1[pcode_src]]
            if suburb_src in suburbs_lookup:
                return 1
            else:
                return 0
        except KeyError:
            return 'invalid postcode'

In [32]:
df['pcode_check'] = df.apply(lambda row: pcode_valid(row), axis=1)

In [33]:
display(df)

,ssn,first_name,middle_name,last_name,gender,age_at_consultation,birth_date,medicare_number,street_address,suburb,state,postcode,phone,email,marital_status,height,weight,bmi,blood_pressure,cholesterol_level,smoking_status,clinical_notes,consultation_timestamp,education,occupation,salary,credit_card_number,years_of_experience,employment_timestamp,medical_rec_id,education_rec_id,pcode_check
0,a100050102,justin,elton,verga,m,36,1980-06-07,8250 99889 1 2,40 marcus clarke street canberra house,canberra city,act,2601,02 5920 4201,verga.justin@mail.com,married,191,67,18,76,138,0,urinary-tract-infection,2016-01-26 06:44:00+00:00,certificate-iv,other-worker,78359,7426 3365 3452 2350,21,2020-04-15 13:24:00+00:00,rec-86693,r15002000,0
1,a100091149,christopher,jerome,herring,m,21,1995-10-20,2939 34141 2 3,17 inner harbour drive marina palms,patterson lakes,vic,3197,03 6566 8753,christopher.herring@mail.com.au,married-de-facto,174,108,35,77,160,0,back-pain,2016-07-18 13:58:00+00:00,certificate-iii,ict-technician,104897,7356 0165 1634 7804,6,2020-06-13 01:21:00+00:00,rec-10443,r90040119,1
2,a100222723,billy,david,ross,m,15,1998-05-17,5417 95547 2 3,41 cavenagh street plaza bldg,darwin,nt,0800,08 4559 6907,ross94@mail.com.au,not-applicable,176,73,23,79,191,0,insomnia,2013-03-26 21:59:00+00:00,secondary-school,machinary-driver,14251,5837 2758 2366 5640,3,2020-04-05 06:45:00+00:00,rec-71086,r23272200,1
3,a100248307,howard,stephen,turner,m,36,1985-01-09,6472 28876 1 1,1351 sidonia road balkebah,baynton,vic,3444,03 4442 1139,howard.turner@gmail.com,married,187,128,36,74,132,0,insomnia,2020-07-23 06:51:00+00:00,certificate-ii,personal-services-worker,78420,3951 5724 9004 2669,15,2019-09-27 12:53:00+00:00,rec-56402,r20407803,1
4,a100317978,keith,andrew,darr,m,14,2002-07-01,4693 25183 1 2,4 morris way jerrabung,bunyip,vic,3815,03 8293 5481,darr60@mail.com.au,not-applicable,157,45,18,70,139,0,shuffling-gait,2016-08-17 21:12:00+00:00,senior-secondary-school,casual-worker,6909,8247 4592 3961 9751,1,2020-01-24 07:11:00+00:00,rec-674,r97030187,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16060,i199840951,evan,maston,bruno,m,23,1996-04-03,4352 69591 2 3,42 borenore lane kenvale,borenore,nsw,2800,02 1588 8711,bruno50@hotmail.com,married,194,75,19,74,181,0,depressed-mood,2018-12-19 16:05:00+00:00,advance-diploma,art-media-professional,103367,4496 1676 9466 2010,5,2020-04-02 07:41:00+00:00,rec-96364,r40995189,1
16061,i199919157,jillian,erin,perell,f,33,1984-11-22,3005 36572 1 1,3 collins close masonic vlge,lake haven,nsw,2263,02 9404 9177,perell.jillian@mail.com,married,199,84,21,76,140,0,flu,2017-08-18 00:14:00+00:00,advance-diploma,education-professional,112247,9392 1545 1088 9225,17,2020-02-16 06:38:00+00:00,rec-6949,r19995719,1
16062,i199940217,martha,renee,jackson,f,15,2000-05-11,8818 54716 2 2,22 mitchell street central specialist clinic,darwin,nt,0800,08 6383 0084,jackson.martha@mail.com.au,not-applicable,177,61,19,65,218,0,leg-injury,2015-07-11 01:52:00+00:00,certificate-iii,automotive-worker,91101,8278 6665 0763 8863,1,2020-05-20 04:18:00+00:00,rec-2966,r97914920,1
16063,i199953012,thomas,joseph,petty,m,27,1989-04-22,9821 42905 2 2,20 dalley street st vincents private hospital,lismore,nsw,2480,02 5323 4883,petty88@aol.com,widowed,176,140,45,78,266,1,cataract-right-eye-(worsening),2016-06-08 08:10:00+00:00,advance-diploma,ict-professional,87913,5666 8118 7243 3499,12,2020-04-17 18:38:00+00:00,rec-67404,r91953920,1


In [36]:
df.to_csv('final_export_pcode_check.csv', index=False)